# Crypto Returns vs Risk

<font size=3> In this post, we'll examine cryptocurrencies by comparing their **annualized returns vs risks** </font>

<font size=3> We'll do the following </font>

<font size=3> - Get top **30** tokens historical price </font>

<font size=3> - Compute **returns** and visualize </font>

<font size=3> - Compute **risks** (volatility) and visualize </font>

<font size=3> - Compute **returns/risks** ratio and visualize </font>

<font size=3> - Visualize **buckets** of tokens based on high/low returns vs high/low risks </font>


<font size=3> **Please Note: This article is only for educational purposes, not an investment advice** </font>

In [1]:
#!pip install -U pycoingecko

## [1] Data Pull

<font size=3> We use `pycoingecko` package and use the same util files we saw in this [data collection](https://web3ml.xyz/blogs/data-collection) article <font>

In [2]:
import pandas as pd
import numpy as np
import datetime
import time
import altair as alt
from pycoingecko import CoinGeckoAPI
import tqdm
import math
import warnings

warnings.filterwarnings("ignore")
pd.set_option("display.max_rows", 1000)
alt.renderers.enable("default")
cg = CoinGeckoAPI()

<font size=3> Fetch top 100 coin names from `coingecko` </font>

In [3]:
df_top100_coins = pd.DataFrame(cg.get_coins_markets(vs_currency="USD"))
top10 = df_top100_coins["id"].values[0:10]
print(f'Top 10 coins are : {"| ".join(top10)}')

Top 10 coins are : bitcoin| ethereum| tether| binancecoin| usd-coin| ripple| binance-usd| dogecoin| cardano| solana


----
## [2] Data Filter

Here we filter for 

(a) top 30 coins and 

(b) remove stable coins since their returns will be very different compared to native crypto coins

In [4]:
top30_coins = df_top100_coins["id"].values[:30]
print(f'Top 30 coins are : {"| ".join(top30_coins)}')

Top 30 coins are : bitcoin| ethereum| tether| binancecoin| usd-coin| ripple| binance-usd| dogecoin| cardano| solana| matic-network| polkadot| staked-ether| shiba-inu| tron| dai| uniswap| okb| avalanche-2| wrapped-bitcoin| litecoin| cosmos| chainlink| leo-token| ethereum-classic| crypto-com-chain| algorand| ftx-token| monero| stellar


<font size=3> List of stable coins excluded from analysis </font>

`tether`
`usd-coin`
`binance-usd`
`dai`
    

In [5]:
coins = list(set(top30_coins) - set(["tether", "usd-coin", "binance-usd", "dai"]))
print(f"Total coins that we'll analyze: {len(coins)}")

Total coins that we'll analyze: 26


 <font size=3> We use the below util functions to get historical price data for the top 100 coins fetched above </font>

 These util functions are reused from [data collection](https://web3ml.xyz/blogs/data-collection) article 

In [6]:
def get_date_information(value):
    """Function to get the datetime from epoch time"""
    return datetime.datetime.fromtimestamp(value[0] / 1000)


def get_value_from_list(value):
    """Fucntion to get the value from response"""
    return value[1]


def get_historical_data_for_coin(coin_name, n_days):
    """Function to get the hisrotical data for one single coin"""
    time.sleep(0.1)
    coin_data = cg.get_coin_market_chart_by_id(
        id=coin_name, vs_currency="usd", days=n_days
    )
    coin_df = pd.DataFrame(coin_data)
    coin_df = coin_df.iloc[:-1]  # Remove today
    coin_df["date"] = coin_df["prices"].apply(lambda x: get_date_information(x))
    coin_df["price"] = coin_df["prices"].apply(lambda x: get_value_from_list(x))
    coin_df["coin_name"] = coin_name
    coin_df.drop(["prices", "total_volumes", "market_caps"], axis=1, inplace=True)
    return coin_df

<font size=3> As an axample, we input `bitcoin` to the above function and see the data below that we get from coingecko

In [7]:
get_historical_data_for_coin("bitcoin", 1000).head()

,date,price,coin_name
0,2020-02-12 05:30:00,10226.925769,bitcoin
1,2020-02-13 05:30:00,10328.897566,bitcoin
2,2020-02-14 05:30:00,10221.260309,bitcoin
3,2020-02-15 05:30:00,10320.157141,bitcoin
4,2020-02-16 05:30:00,9894.136124,bitcoin


<font size=3> Fetch the above historical price data for top 30 coins </font>

In [8]:
coins_df = pd.concat(
    get_historical_data_for_coin(coin_name=coin_id, n_days=1000) for coin_id in tqdm.tqdm(coins)
)
coins_df["date"] = coins_df["date"].dt.date
print("Sample Data")
coins_df.head()

100%|███████████████████████████████████████████| 26/26 [02:16<00:00,  5.25s/it]

Sample Data


,date,price,coin_name
0,2020-02-12,0.003053,dogecoin
1,2020-02-13,0.003051,dogecoin
2,2020-02-14,0.003033,dogecoin
3,2020-02-15,0.003102,dogecoin
4,2020-02-16,0.002960,dogecoin


<font size=3> Cast data such that each coin's return is a column </font>

In [9]:
coins_df = pd.pivot_table(data=coins_df, index=["date"], columns="coin_name")
coins_df.columns = [col[1] for col in coins_df.columns]
coins_df.head(5)

,algorand,avalanche-2,binancecoin,bitcoin,cardano,chainlink,cosmos,crypto-com-chain,dogecoin,ethereum,...,okb,polkadot,ripple,shiba-inu,solana,staked-ether,stellar,tron,uniswap,wrapped-bitcoin
date,,,,,,,,,,,,,,,,,,,,,
2020-02-11,0.361979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-02-12,0.369179,NaN,25.673327,10226.925769,0.062890,3.971540,5.114089,0.062017,0.003053,236.785349,...,5.558884,NaN,0.280596,NaN,NaN,NaN,0.074029,0.022768,NaN,10214.020823
2020-02-13,0.389243,NaN,26.420489,10328.897566,0.067292,4.004469,5.148892,0.063962,0.003051,264.032768,...,5.910529,NaN,0.300625,NaN,NaN,NaN,0.079962,0.023855,NaN,10318.176379
2020-02-14,0.348289,NaN,25.403531,10221.260309,0.068365,3.873599,4.901170,0.066034,0.003033,267.670445,...,5.894890,NaN,0.323237,NaN,NaN,NaN,0.083081,0.023869,NaN,10289.409462
2020-02-15,0.366205,NaN,26.237615,10320.157141,0.070329,4.409025,5.099030,0.066701,0.003102,284.231891,...,6.431447,NaN,0.333282,NaN,NaN,NaN,0.086026,0.026080,NaN,10420.328200


----
## [3] Daily returns

- In this section, we compute daily returns as the percentage change in coin price value between current date and previous date

- Visualize the ditribution of daily returns for all these coins

In [10]:
returns_df = coins_df.pct_change()
returns_df.tail()

,algorand,avalanche-2,binancecoin,bitcoin,cardano,chainlink,cosmos,crypto-com-chain,dogecoin,ethereum,...,okb,polkadot,ripple,shiba-inu,solana,staked-ether,stellar,tron,uniswap,wrapped-bitcoin
date,,,,,,,,,,,,,,,,,,,,,
2022-11-03,0.007440,-0.040641,-0.013652,-0.015943,-0.037068,-0.034166,-0.057238,-0.022640,-0.102560,-0.037801,...,-0.018818,-0.034441,-0.027764,-0.084900,-0.045056,-0.037442,-0.023386,-0.020282,-0.004082,-0.015636
2022-11-04,0.015829,0.007150,0.029111,0.002377,0.007973,0.036287,0.013570,0.009878,-0.042206,0.006888,...,0.238041,0.027676,0.008243,-0.004888,0.001242,0.005952,0.009372,0.003032,-0.023377,0.002651
2022-11-05,0.121218,0.074327,0.070882,0.046480,0.082802,0.135742,0.114647,0.058673,0.028668,0.073855,...,0.031912,0.101183,0.103173,0.068758,0.096080,0.074235,0.060922,0.031074,0.087288,0.046769
2022-11-06,0.025658,0.011491,-0.012020,0.006273,0.012811,-0.009544,0.024363,0.036154,-0.013200,-0.010574,...,0.075787,-0.007183,-0.018023,-0.005680,0.086715,-0.010174,-0.015979,-0.001392,0.019733,0.006341
2022-11-07,0.000000,-0.066123,-0.022399,-0.017666,-0.053596,-0.062934,0.000000,-0.023699,-0.077918,-0.033107,...,-0.013481,-0.028591,-0.044921,0.000000,0.000000,-0.035158,-0.038599,-0.012790,-0.079132,-0.016561


<font size=3> **Daily Returns Distribution** </font>

In [11]:
alt.data_transformers.disable_max_rows()

source = returns_df.copy(deep=True)
source["Date"] = source.index
source = pd.melt(source, id_vars="Date")


chart = (
    alt.Chart(source[["variable", "value"]])
    .mark_bar()
    .encode(
        x=alt.X(
            "value:Q",
            axis=alt.Axis(title="Daily Returns Binned", format=".0%"),
            scale=alt.Scale(zero=False),
            bin=alt.Bin(maxbins=20),
        ),
        y=alt.Y("count():Q", axis=alt.Axis(title="Bin Count")),
        color=alt.Color("variable:N"),
    )
    .properties(width=130, height=130)
)

alt.ConcatChart(
    concat=[
        chart.transform_filter(alt.datum.variable == value).properties(title=value)
        for value in sorted(source[["variable", "value"]].variable.unique())
    ],
    columns=4,
).configure_title(
    fontSize=20, font="Courier", anchor="middle", color="gray", align="left"
).resolve_axis(
    x="independent", y="independent"
).resolve_scale(
    x="independent", y="independent"
)

alt.ConcatChart(...)

**Inference**

- Daily Returns for all tokens vary from -50% to 80% for all coins except dogecoin and shiba-inu


- For shiba-inu it ranges from -200% to to 3500%

----


## [4] Annualized Returns

- Compute annualized returns based on the daily returns calculated.
- Since they are traded daily we use 365 days to get annualized returns

In [12]:
annualized_returns_df = pd.DataFrame(returns_df.mean() * 365)
annualized_returns_df.reset_index(inplace=True)
annualized_returns_df.columns = ["coin_name", "annualized_returns"]
annualized_returns_df.sort_values("annualized_returns", ascending=True, inplace=True)
annualized_returns_df.head(5)

,coin_name,annualized_returns
13,litecoin,0.494008
3,bitcoin,0.534063
25,wrapped-bitcoin,0.548287
15,monero,0.729486
12,leo-token,0.731193


In [13]:
source = pd.concat([annualized_returns_df.head(15), annualized_returns_df.tail(15)])
source = annualized_returns_df

bars = (
    alt.Chart(source, title="Annualized Returns for Coins")
    .mark_bar()
    .encode(
        x=alt.X("coin_name", sort=None, axis=alt.Axis(labelAngle=-45)),
        y=alt.Y("annualized_returns:Q", sort=None, axis=alt.Axis(title="Annualized Returns %", format="%")),
        color=alt.condition(
            alt.datum.annualized_returns > 0,
            alt.value("limegreen"),  # The positive color
            alt.value("orange"),  # The negative color
        )
    )).properties(width=900)


text = bars.mark_text(align="center", baseline="bottom",).encode(
    text=alt.Text("annualized_returns:Q", format=".0%"),
    color=alt.condition(
        alt.datum.annualized_returns > 0,
        alt.value("green"),  # The positive color
        alt.value("red"),  # The negative color
    ),
)

(bars + text).properties(width=900).configure_title(
    fontSize=20, font="Courier", anchor="middle", color="gray", align="left")

alt.LayerChart(...)

**Inference**

- Annualized returns for the top 26 tokens vary from *~50% to ~2500%*


- The top 5 tokens with highest annualized returns are `avalance-2`, `solana`, `matic`, `dogecoin`, `shiba-inu`


- The bottom 5 tokens based on annualized returns are `litecoin`, `bitcoin`, `w-bitcoin`, `monero`, `leo`

----

## [5] Annualized Market Volatility

- We measure Volatility as the variance / `standard deviation` of the stock returns 
- Annualized volatility is calculated as `std_dev` * `sqrt(time period)` since *variance is proportional to time*
- See [this](https://www.macroption.com/why-is-volatility-proportional-to-square-root-of-time/) for more details

In [14]:
annualized_volatility_df = pd.DataFrame(returns_df.std() * math.sqrt(365))
annualized_volatility_df.reset_index(inplace=True)
annualized_volatility_df.columns = ["coin_name", "annualized_volatility"]
annualized_volatility_df.sort_values(
    "annualized_volatility", ascending=True, inplace=True
)

In [15]:
source = annualized_volatility_df
median_volatility = np.median(source["annualized_volatility"])
bars = (
    alt.Chart(source, title="Annualized Volatility for coins ")
    .mark_bar()
    .encode(
        y=alt.Y("coin_name", sort=None, axis=alt.Axis(labelAngle=-45)),
        x=alt.X(
            "annualized_volatility:Q", sort=None, axis=alt.Axis(title= "Annualized Volatility % - Std Dev of Returns", format=".0%")
        ),
        color=alt.condition(
            alt.datum.annualized_volatility > median_volatility,
            alt.value("orange"),  # The positive color
            alt.value("green"),  # The negative color
        ),
    )
    .properties(width=900)
)


text = bars.mark_text(align="center", baseline="bottom", dx=20, dy=5).encode(
    text=alt.Text("annualized_volatility:Q", format=".0%"),
)

(bars + text).properties(width=900).configure_title(
    fontSize=20, font="Courier", anchor="middle", color="gray", align="left")

alt.LayerChart(...)

**Inference**

- Median annualized volatility is around 119%


- Tokens with annualized volatility less than median are colored in <span style="color:green">green</span> and the others in <span style="color:orange">orange </span>


- In general tokens with higher daily returns are also more volatile, but there are some exceptions like `algorand`, `ripple` (low returns, high volatility) `binance coin` (high returns, low volatility)  etc. 


- The reward (return) to risk (volatility) ratio will help us understand further about such exceptions
----

## [6] Return/Risk Ratio

In this section, we'll compute return/volatility ratio and compare tokens

In [16]:
df = pd.merge(annualized_returns_df, annualized_volatility_df, on="coin_name")
df["reward_by_volatility_ratio"] = (
    df["annualized_returns"] / df["annualized_volatility"]
)
df.sort_values("reward_by_volatility_ratio", ascending=False, inplace=True)

In [17]:
source = df
bars = (
    alt.Chart(source, title="Return vs Volatility Ratio")
    .mark_bar()
    .encode(
        x=alt.X("coin_name", sort=None, axis=alt.Axis(labelAngle=-45)),
        y=alt.Y(
            "reward_by_volatility_ratio:Q",
            sort=None,
            axis=alt.Axis(title="Returns / Volatility Ratio", format=".2f"),
        ),
        color=alt.condition(
            alt.datum.reward_by_volatility_ratio > 1,
            alt.value("limegreen"),  # The positive color
            alt.value("orange"),  # The negative color
        ),
    )
    .properties(width=900)
)


text = bars.mark_text(align="center", baseline="bottom",).encode(
    text=alt.Text("reward_by_volatility_ratio:Q", format=".2f"),
    color=alt.condition(
        alt.datum.reward_by_volatility_ratio > 0,
        alt.value("green"),  # The positive color
        alt.value("red"),  # The negative color
    ),
)

(bars + text).properties(width=900).configure_title(
    fontSize=20, font="Courier", anchor="middle", color="gray", align="left")

alt.LayerChart(...)

**Inference**

- 11 coins (out of top 26) coins have a `return by volatility ratio` greater than 1 signifying that their annualized returns is more than the volatility we see in their returns. The remaining have annualized returns lesser than the volatility observed in their returns


- Top 3 coins with high `return by volatility ratio` are `solana`, `matic`, `binance coin` meaning their return as

----

## [7] Bringing it all together

In this section, we'll visualize all three things we analyzed earlier namely

- Annualized Returns
- Annualized Volatility
- Return / Volatility Ratio

into quadrants of high/low returns vs high/low volatility 

In [18]:
source = df[df["coin_name"] != "shiba-inu"]

source["reward_by_vol_gt_1"] = np.where(
    source["reward_by_volatility_ratio"] > 1,
    "High Return/Volatility",
    "Low Return/Volatility",
)

domain = ["High Return/Volatility", "Low Return/Volatility"]
range_ = ["limegreen", "orange"]

scatter = (
    alt.Chart(
        source,
        title=f"Annualized Returns vs Volatility",
    )
    .mark_point()
    .encode(
        x=alt.X("annualized_returns", axis=alt.Axis(format=".0%")),
        y=alt.Y("annualized_volatility", axis=alt.Axis(format=".0%")),
        color=alt.Color(
            "reward_by_vol_gt_1", scale=alt.Scale(domain=domain, range=range_)
        ),
        tooltip=[
            alt.Tooltip("coin_name"),
            alt.Tooltip("annualized_returns", format=".0%"),
            alt.Tooltip("annualized_volatility", format=".0%"),
            alt.Tooltip("reward_by_volatility_ratio", format=".2f"),
        ],
    )
).properties(height=600, width=800)

vline = (
    alt.Chart(source)
    .mark_rule(color="blue")
    .encode(y="median(annualized_volatility):Q")
)

hline = (
    alt.Chart(source)
    .mark_rule(color="blue")
    .encode(
        x="median(annualized_returns):Q",
    )
)

text = scatter.mark_text(align="left", baseline="middle", dx=7).encode(
    text="coin_name",
)

(scatter + vline + hline + text).configure_axis(
    grid=False, domain=False
).configure_view(strokeWidth=0).configure_title(
    fontSize=20, font="Courier", anchor="middle", color="gray", align="left")

alt.LayerChart(...)

- Horizontal line indicates median volatility observed across these 26 tokens
- Vertical line indicates median returns observed across these 26 tokens

- Color indicates having high return/volatility ratio (green) and low return/volatility ratio (orange)

**Top Left Quadrant** signifies coins with `low returns/high volatility`


**Top Right Quadrant** signifies coins with `high returns/high volatility`


**Bottom Left Quadrant** signifies coins with `low returns/low volatility`


**Botttom Right Quadrant** signifies coins with `high returns/low volatility` (*Quadrant with all tokens in green i.e: high return/volatility ratio*)

-----

<ins>**To summarize the top 10 coins (after removing for stable coins)**<ins>
    


`bitcoin`     - <span style="color:red">low returns </span> | <span style="color:green">low volatility </span> | <span style="color:red">low return/risk ratio </span>

`ethereum`    - <span style="color:green">high returns </span> | <span style="color:green">low volatility </span> | <span style="color:green">high return/risk ratio </span>

`binancecoin` - <span style="color:green">high returns </span> | <span style="color:green">low volatility </span> | <span style="color:green">high return/risk ratio </span>

`ripple`      - <span style="color:red">low returns </span> | <span style="color:red">high volatility </span> | <span style="color:red">low return/risk ratio </span>

`dogecoin`    - <span style="color:green">high returns </span> | <span style="color:red">high volatility </span> | <span style="color:green">high return/risk ratio </span>

`cardano`     - <span style="color:green">high returns </span> | <span style="color:green">low volatility </span> | <span style="color:green">high return/risk ratio </span>

`solana`      - <span style="color:green">high returns </span> | <span style="color:red">high volatility </span> | <span style="color:green">high return/risk ratio </span>
    
`matic-network` - <span style="color:green">high returns </span> | <span style="color:red">high volatility </span> | <span style="color:green">high return/risk ratio </span>
    
`polkadot` - <span style="color:green">high returns </span> | <span style="color:red">high volatility </span> | <span style="color:red">low return/risk ratio </span>
    
`shiba-inu` - <span style="color:green">high returns </span> | <span style="color:red">high volatility </span> | <span style="color:red">high return/risk ratio </span>




With this we come to the end of this post analyzing returns vs volatility. Hope you enjoyed it. Do let us know your valualable feedback and other things you'd have liked to include when doing such coin market performance analysis. Until next time, take care!

-----